In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, when, substring, datediff, to_date, row_number, count

drivers = spark.read.csv("s3://columbia-gr5069-main/raw/drivers.csv", header=True)
results = spark.read.csv("s3://columbia-gr5069-main/raw/results.csv", header=True)
races = spark.read.csv("s3://columbia-gr5069-main/raw/races.csv", header=True)
pit_stops = spark.read.csv('s3://columbia-gr5069-main/raw/pit_stops.csv', header=True)

#### Question 1: What was the average time each driver spent at the pit stop for each race?

In [0]:
avg_pit_time = pit_stops.groupBy("raceId", "driverId").agg(avg("milliseconds").alias("avg_pit_time_ms"))
display(avg_pit_time)

#### Question 2: Rank the average time spent at the pit stop in order of who won each race

In [0]:
winners = results.filter(col("positionOrder") == 1).select("raceId", "driverId").withColumnRenamed("driverId", "winner_driverId")

joined = avg_pit_time.join(winners, "raceId")
winner_pit_times = joined.filter(col("driverId") == col("winner_driverId")).orderBy("avg_pit_time_ms")

display(winner_pit_times)